In [26]:
import os
import pandas as pd
import numpy as np
import random
import torch
from torch.utils.tensorboard import SummaryWriter
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [13]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(21, 21),
            nn.ReLU(),
            nn.Linear(21, 21),
            nn.ReLU(),
            nn.Linear(21, 21),
            nn.ReLU(),
            nn.Linear(21, 10),
            nn.ReLU(),
            nn.Linear(10, 3),
        )

    def forward(self, x):
        out = self.linear_relu_stack(x)
        return out

In [29]:
# Hyper-parameters 
seed = 42
num_epochs = 5
batch_size = 512
learning_rate = 3e-4
train_size = 0.9

random.seed(seed)

In [ ]:
X = pd.read_csv('data/train/X.csv').reset_index(drop=True)
y = pd.read_csv('data/train/y.csv').reset_index(drop=True)

train_inds = random.sample(list(X.index.values), int(train_size*len(X.index)))

train_dataset = TensorDataset(
    torch.tensor(X.iloc[train_inds].values.astype(np.float32)),
    torch.tensor(y.iloc[train_inds].values.astype(np.float32))
)
test_dataset = TensorDataset(
    torch.tensor(X[~X.index.isin(train_inds)].values.astype(np.float32)),
    torch.tensor(y[~y.index.isin(train_inds)].values.astype(np.float32))
)

In [32]:
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = False)

In [36]:
model = NeuralNetwork().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

writer = SummaryWriter()

In [37]:
# Train the model
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (products, labels) in enumerate(train_loader):  
        # origin shape: [100, 1, 28, 28]
        # resized: [100, 784]
        
        products = products.to(device)
        labels = labels.to(device)
        
        # Forward pass and loss calculation
        outputs = model(products)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if (i+1) % 1000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
            
    writer.add_scalar("Loss/train", loss, epoch)

writer.flush()

Epoch [1/5], Step [1000/7537], Loss: 1.0958
Epoch [1/5], Step [2000/7537], Loss: 1.0975
Epoch [1/5], Step [3000/7537], Loss: 1.0966
Epoch [1/5], Step [4000/7537], Loss: 1.0915
Epoch [1/5], Step [5000/7537], Loss: 1.0932
Epoch [1/5], Step [6000/7537], Loss: 1.0971
Epoch [1/5], Step [7000/7537], Loss: 1.0996
Epoch [2/5], Step [1000/7537], Loss: 1.0960
Epoch [2/5], Step [2000/7537], Loss: 1.0939
Epoch [2/5], Step [3000/7537], Loss: 1.0987
Epoch [2/5], Step [4000/7537], Loss: 1.0926
Epoch [2/5], Step [5000/7537], Loss: 1.0959
Epoch [2/5], Step [6000/7537], Loss: 1.0954
Epoch [2/5], Step [7000/7537], Loss: 1.0928
Epoch [3/5], Step [1000/7537], Loss: 1.0994
Epoch [3/5], Step [2000/7537], Loss: 1.0985
Epoch [3/5], Step [3000/7537], Loss: 1.0990
Epoch [3/5], Step [4000/7537], Loss: 1.0963
Epoch [3/5], Step [5000/7537], Loss: 1.0967
Epoch [3/5], Step [6000/7537], Loss: 1.0945
Epoch [3/5], Step [7000/7537], Loss: 1.0971
Epoch [4/5], Step [1000/7537], Loss: 1.0984
Epoch [4/5], Step [2000/7537], L

In [16]:
products

tensor([[0.9298, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [   nan,    nan,    nan,  ...,    nan,    nan,    nan],
        [0.7692, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.7926, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.9197, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0502, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])